In [ ]:
import sys
sys.version

'3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]'

In [ ]:
import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [ ]:
from huggingface_hub import login

# Hugging Face API 토큰 입력
login(token="")

# from huggingface_hub import whoami
# print(whoami())

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import requests
import time
import ast
import re
import os
import json
import random
import sklearn
# from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
# 지표를 하나만 설정할 경우
from sklearn.model_selection import cross_val_score
# 지표를 하나 이상 설정할 경우
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold


# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score, roc_curve

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


# 차원축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 군집화
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth

# 시간 측정을 위한 시간 모듈
import datetime
from tqdm import tqdm

# 형태소 벡터를 생성하기 위한 라이브러리
from sklearn.feature_extraction.text import CountVectorizer

# 형태소 벡터를 학습 벡터로 변환한다.
from sklearn.feature_extraction.text import TfidfTransformer

# 한국어 형태소 분석
# from konlpy.tag import Okt, Hannanum, Kkma, Mecab, Komoran

# 저장
import pickle

# torch
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, AutoModel
from torch.optim import AdamW
from torch.utils.data import WeightedRandomSampler
from transformers import get_scheduler
import argparse
from transformers import BertForMaskedLM



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_train.csv")
val_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_val.csv")
test_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_test.csv")

# rs_sample = pd.read_csv('/content/drive/My Drive/25.1H Thesis/just_data_250423f.csv')


In [ ]:
# 모델
# roberta-small
# roberta-base
# monologg/distilkobert
# lighthouse/mdeberta-v3-base-kor-further

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
model_class = AutoModelForSequenceClassification

model_name = "klue/roberta-base"

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
result_dir = f"/content/drive/My Drive/cmf/"

gradient

In [ ]:

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "pure_grd"

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []
    grad_norms_log = []
    epoch_stats_log = []  # 에폭별 통계

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        epoch_grad_norms = []

        for step, batch in enumerate(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()
            total_norm = 0
            for p in model.parameters():
                if p. grad is not None:
                     param_norm = p.grad.data.norm(2)
                     total_norm += param_norm.item() ** 2

            total_norm = total_norm ** 0.5
            epoch_grad_norms.append(total_norm)
            grad_norms_log.append((epoch, step, total_norm))

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

        epoch_avg_norm = np.mean(epoch_grad_norms) if epoch_grad_norms else 0
        epoch_std_norm = np.std(epoch_grad_norms) if epoch_grad_norms else 0
        epoch_stats_log.append((epoch, epoch_avg_norm, epoch_std_norm))

        print(f"Epoch {epoch+1} - Gradient norm stats: Avg: {epoch_avg_norm:.4f}, Std: {epoch_std_norm:.4f}")

        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")

    # 그래디언트 노름 로그 저장
    grad_norms_df = pd.DataFrame(grad_norms_log, columns=['epoch', 'step', 'batch_norm'])
    grad_norms_df.to_csv(f"{result_dir}grad_norms_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8-sig')

    epoch_stats_df = pd.DataFrame(epoch_stats_log, columns=['epoch', 'avg_norm', 'std_norm'])
    epoch_stats_df.to_csv(f"{result_dir}epoch_grad_stats_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8-sig')

    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    train_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_train.csv")
    val_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_val.csv")
    test_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_test.csv")

    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=1,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/1 =====


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at lighthouse/mdeberta-v3-base-kor-further and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.6373 | Acc: 0.6925 | F1: 0.0000 | Precision: 0.0000 | Recall: 0.0000 | AUROC: 0.5192 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Epoch 1 - Gradient norm stats: Avg: 0.7971, Std: 1.9170
Run 1, Epoch 1 성능 평가:
Loss: 0.0100 | Acc: 0.9974 | F1: 0.9957 | Precision: 0.9915 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9800
Run 1: New best model found with combined score: 0.9800
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9800 (weight: 1.00)
Epoch 2 - Gradient norm stats: Avg: 0.0820, Std: 0.5162
Run 1, Epoch 2 성능 평가:
Loss: 0.0068 | Acc: 0.9980 | F1: 0.9968 | Precision: 0.9936 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9986
Run 1: New best model found with combined score: 0.9986
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9986 (weight: 1.00)
Epoch 3 - Gradient norm stats: Avg: 0.1354, Std: 0.8432
Run 1, Epoch 3 성능 평가:
Loss: 0.0052 | Acc: 0.9987 | F1: 0.9979 | Precision: 0.9957 | R

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at lighthouse/mdeberta-v3-base-kor-further and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.1993 | Acc: 0.9536 | F1: 0.9212 | Precision: 0.9983 | Recall: 0.8552 | AUROC: 0.9992 | TPR@FPR=0.01%: 0.4734
소요 시간: 102.07분


In [ ]:
# static

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "static_grd" #static

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }

def re_weighting_sequence(target_series, mode='static'):
    wrong_cnt = np.sum(target_series)*1.0
    right_cnt = len(target_series)*1.0 - wrong_cnt
    if wrong_cnt == 0:
        wrong_cnt = 1
    if mode == 'adaptive':
        res_series = [right_cnt*1.0 / wrong_cnt if e == 1 else 1 for e in target_series]
    elif mode == 'static':
        res_series = [2 if e == 1 else 1 for e in target_series]

    return res_series

def wrong_first_dataloader(model, tokenized_datasets, batch_size=16, num_cls=2, current_epoch=1, device=torch.device("cpu")):
    if current_epoch == 1:
        return DataLoader(tokenized_datasets, shuffle=True, batch_size=batch_size)

    eval_dataloader = DataLoader(tokenized_datasets, shuffle=False, batch_size=1)

    all_preds = []
    all_labels = []
    right_wrong_list = []

    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )


        predictions = torch.argmax(outputs.logits, dim=-1)
        all_preds.append(predictions.item())
        all_labels.append(batch["labels"].item())

        right_wrong_list.append(1 if predictions.item() != batch["labels"].item() else 0)

    # 샘플링 가중치 설정
    weights = re_weighting_sequence(right_wrong_list)

    # 가중치 길이가 맞는지 확인
    if len(weights) != len(tokenized_datasets):
        print(f"Warning: weights length {len(weights)} does not match dataset size {len(tokenized_datasets)}")

    sampler = WeightedRandomSampler(weights=weights, num_samples=len(tokenized_datasets), replacement=True)

    return DataLoader(tokenized_datasets, batch_size=batch_size, sampler=sampler)




def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)


def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)
    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls)
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')


    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []
    grad_norms_log = []
    epoch_stats_log = []  # 에폭별 통계

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화
        epoch_grad_norms = []

        train_dataloader_current = wrong_first_dataloader(
            model,
            train_dataloader.dataset,
            batch_size=train_dataloader.batch_size,
            num_cls=num_cls,
            current_epoch=epoch+1,
            device=device
        )

        for step, batch in enumerate(train_dataloader_current) :
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()
            total_norm = 0

            for p in model.parameters():
                if p.grad is not None:
                     param_norm = p.grad.data.norm(2)
                     total_norm += param_norm.item() ** 2

            total_norm = total_norm ** 0.5
            epoch_grad_norms.append(total_norm)
            grad_norms_log.append((epoch, step, total_norm))

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

        epoch_avg_norm = np.mean(epoch_grad_norms) if epoch_grad_norms else 0
        epoch_std_norm = np.std(epoch_grad_norms) if epoch_grad_norms else 0
        epoch_stats_log.append((epoch, epoch_avg_norm, epoch_std_norm))

        print(f"Epoch {epoch+1} - Gradient norm stats: Avg: {epoch_avg_norm:.4f}, Std: {epoch_std_norm:.4f}")

        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"


        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)



       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")

    # 그래디언트 노름 로그 저장
    grad_norms_df = pd.DataFrame(grad_norms_log, columns=['epoch', 'step', 'batch_norm'])
    grad_norms_df.to_csv(f"{result_dir}grad_norms_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8-sig')

    epoch_stats_df = pd.DataFrame(epoch_stats_log, columns=['epoch', 'avg_norm', 'std_norm'])
    epoch_stats_df.to_csv(f"{result_dir}epoch_grad_stats_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8-sig')


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    train_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_train.csv")
    val_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_val.csv")
    test_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_test.csv")


    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=1,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/1 =====


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.7217 | Acc: 0.3075 | F1: 0.4704 | Precision: 0.3075 | Recall: 1.0000 | AUROC: 0.6294 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Epoch 1 - Gradient norm stats: Avg: 1.7299, Std: 4.5907
Run 1, Epoch 1 성능 평가:
Loss: 0.1433 | Acc: 0.9556 | F1: 0.9327 | Precision: 0.8739 | Recall: 1.0000 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.6864
Run 1: New best model found with combined score: 0.6864
  - AUROC: 0.9998 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.6864 (weight: 1.00)
Epoch 2 - Gradient norm stats: Avg: 1.3237, Std: 6.9470
Run 1, Epoch 2 성능 평가:
Loss: 0.0017 | Acc: 0.9991 | F1: 0.9986 | Precision: 0.9986 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9986
Run 1: New best model found with combined score: 0.9986
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9986 (weight: 1.00)
Epoch 3 - Gradient norm stats: Avg: 0.0508, Std: 0.5319
Run 1, Epoch 3 성능 평가:
Loss: 0.0058 | Acc: 0.9989 | F1: 0.9982 | Precision: 1.0000 | R

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 1.0368 | Acc: 0.8340 | F1: 0.6457 | Precision: 0.9990 | Recall: 0.4771 | AUROC: 0.9971 | TPR@FPR=0.01%: 0.4331
소요 시간: 60.64분


In [ ]:
# 모델
# roberta-small
# roberta-base
# monologg/distilkobert
# lighthouse/mdeberta-v3-base-kor-further

tokenizer = AutoTokenizer.from_pretrained("lighthouse/mdeberta-v3-base-kor-further")
model_class = AutoModelForSequenceClassification

model_name = "lighthouse/mdeberta-v3-base-kor-further"

tokenizer_config.json:   0%|          | 0.00/424 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [ ]:

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "pure_grd"

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []
    grad_norms_log = []
    epoch_stats_log = []  # 에폭별 통계

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        epoch_grad_norms = []

        for step, batch in enumerate(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()
            total_norm = 0
            for p in model.parameters():
                if p.grad is not None:
                     param_norm = p.grad.data.norm(2)
                     total_norm += param_norm.item() ** 2

            total_norm = total_norm ** 0.5
            epoch_grad_norms.append(total_norm)
            grad_norms_log.append((epoch, step, total_norm))

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

        epoch_avg_norm = np.mean(epoch_grad_norms) if epoch_grad_norms else 0
        epoch_std_norm = np.std(epoch_grad_norms) if epoch_grad_norms else 0
        epoch_stats_log.append((epoch, epoch_avg_norm, epoch_std_norm))

        print(f"Epoch {epoch+1} - Gradient norm stats: Avg: {epoch_avg_norm:.4f}, Std: {epoch_std_norm:.4f}")

        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")

    # 그래디언트 노름 로그 저장
    grad_norms_df = pd.DataFrame(grad_norms_log, columns=['epoch', 'step', 'batch_norm'])
    grad_norms_df.to_csv(f"{result_dir}grad_norms_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8-sig')

    epoch_stats_df = pd.DataFrame(epoch_stats_log, columns=['epoch', 'avg_norm', 'std_norm'])
    epoch_stats_df.to_csv(f"{result_dir}epoch_grad_stats_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8-sig')

    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    train_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_train.csv")
    val_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_val.csv")
    test_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_test.csv")

    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=1,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/1 =====


config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/559M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at lighthouse/mdeberta-v3-base-kor-further and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.6373 | Acc: 0.6925 | F1: 0.0000 | Precision: 0.0000 | Recall: 0.0000 | AUROC: 0.5192 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-


NameError: name 'result_dir' is not defined

## 베이스라인

In [ ]:
# kobert 출력

tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model_class = AutoModelForSequenceClassification
model_name = "monologg/kobert"

tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


tokenization_kobert.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

In [ ]:
result_dir = f"/content/drive/My Drive/cmf/"

In [ ]:
# baseline

result_dir = f"/content/drive/My Drive/cmf/"


safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "pure_2"

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        for step, batch in enumerate(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")

        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():

    train_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_train.csv")
    val_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_val.csv")
    test_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_test.csv")



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.7842 | Acc: 0.3075 | F1: 0.4704 | Precision: 0.3075 | Recall: 1.0000 | AUROC: 0.5772 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.0134 | Acc: 0.9952 | F1: 0.9921 | Precision: 0.9993 | Recall: 0.9850 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9679
Run 1: New best model found with combined score: 0.9679
Run 1, Epoch 2 성능 평가:
Loss: 0.0053 | Acc: 0.9985 | F1: 0.9975 | Precision: 0.9979 | Recall: 0.9971 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9757
Run 1: New best model found with combined score: 0.9757
Run 1, Epoch 3 성능 평가:
Loss: 0.0082 | Acc: 0.9978 | F1: 0.9964 | Precision: 0.9943 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9893
Run 1: New best model found with combined score: 0.9893
Run 1, Epoch 4 성능 평가:
Loss: 0.0105 | Acc: 0.9971 | F1: 0.9954 | Precision: 0.9908 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9964
Run 1: New best model found with combined score: 0.9964
Run 1, Epoch 5 

...

-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.6284 | Acc: 0.6927 | F1: 0.0029 | Precision: 0.6667 | Recall: 0.0014 | AUROC: 0.8445 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.0202 | Acc: 0.9949 | F1: 0.9918 | Precision: 0.9852 | Recall: 0.9986 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.3971
Run 2: New best model found with combined score: 0.3971
Run 2, Epoch 2 성능 평가:
Loss: 0.0029 | Acc: 0.9993 | F1: 0.9989 | Precision: 0.9986 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9993
Run 2: New best model found with combined score: 0.9993
Run 2, Epoch 3 성능 평가:
Loss: 0.0074 | Acc: 0.9974 | F1: 0.9957 | Precision: 0.9915 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9993
Run 2: No improvement. Early stop counter = 1/2
Run 2, Epoch 4 성능 평가:
Loss: 0.0056 | Acc: 0.9985 | F1: 0.9975 | Precision: 0.9950 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9957
Run 2: No improvement. Early stop counter = 2/2
Run 2: Early stopping triggered

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.6837 | Acc: 0.6293 | F1: 0.4715 | Precision: 0.4197 | Recall: 0.5379 | AUROC: 0.6359 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.0149 | Acc: 0.9960 | F1: 0.9936 | Precision: 0.9908 | Recall: 0.9964 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9721
Run 3: New best model found with combined score: 0.9721
Run 3, Epoch 2 성능 평가:
Loss: 0.0063 | Acc: 0.9982 | F1: 0.9971 | Precision: 0.9957 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9964
Run 3: New best model found with combined score: 0.9964
Run 3, Epoch 3 성능 평가:
Loss: 0.0477 | Acc: 0.9886 | F1: 0.9818 | Precision: 0.9642 | Recall: 1.0000 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.4041
Run 3: No improvement. Early stop counter = 1/2
Run 3, Epoch 4 성능 평가:
Loss: 0.0061 | Acc: 0.9989 | F1: 0.9982 | Precision: 0.9979 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9914
Run 3: No improvement. Early stop counter = 2/2
Run 3: Early stopping triggered

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.9725 | Acc: 0.8378 | F1: 0.6573 | Precision: 0.9961 | Recall: 0.4904 | AUROC: 0.9921 | TPR@FPR=0.01%: 0.1262
소요 시간: 118.56분


In [ ]:
# baseline

result_dir = f"/content/drive/My Drive/cmf/"


safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "pure"

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        for step, batch in enumerate(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")

        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():

    train_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_train.csv")
    val_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_val.csv")
    test_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_test.csv")



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.7842 | Acc: 0.3075 | F1: 0.4704 | Precision: 0.3075 | Recall: 1.0000 | AUROC: 0.5772 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.0151 | Acc: 0.9934 | F1: 0.9894 | Precision: 0.9797 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9871
Run 1: New best model found with combined score: 0.9871
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9871 (weight: 1.00)
Run 1, Epoch 2 성능 평가:
Loss: 0.0144 | Acc: 0.9958 | F1: 0.9933 | Precision: 0.9866 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9900
Run 1: New best model found with combined score: 0.9900
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9900 (weight: 1.00)
Run 1, Epoch 3 성능 평가:
Loss: 0.0075 | Acc: 0.9982 | F1: 0.9971 | Precision: 0.9957 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9979
Run 1: New best model found with combined score: 0.9979
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.99

...

-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.6284 | Acc: 0.6927 | F1: 0.0029 | Precision: 0.6667 | Recall: 0.0014 | AUROC: 0.8445 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.0109 | Acc: 0.9967 | F1: 0.9946 | Precision: 0.9964 | Recall: 0.9929 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9929
Run 2: New best model found with combined score: 0.9929
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9929 (weight: 1.00)
Run 2, Epoch 2 성능 평가:
Loss: 0.0039 | Acc: 0.9987 | F1: 0.9979 | Precision: 1.0000 | Recall: 0.9957 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9971
Run 2: New best model found with combined score: 0.9971
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9971 (weight: 1.00)
Run 2, Epoch 3 성능 평가:
Loss: 0.0065 | Acc: 0.9985 | F1: 0.9975 | Precision: 0.9957 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9964
Run 2: No improvement. Early stop counter = 1/2
Run 2, Epoch 4 성능 평가:
Loss: 0.0042 | Acc: 0.9989 | F1: 0.9982 | Pr

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.6837 | Acc: 0.6293 | F1: 0.4715 | Precision: 0.4197 | Recall: 0.5379 | AUROC: 0.6359 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.0078 | Acc: 0.9978 | F1: 0.9964 | Precision: 0.9943 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9914
Run 3: New best model found with combined score: 0.9914
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9914 (weight: 1.00)
Run 3, Epoch 2 성능 평가:
Loss: 0.0030 | Acc: 0.9993 | F1: 0.9989 | Precision: 0.9986 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9943
Run 3: New best model found with combined score: 0.9943
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9943 (weight: 1.00)
Run 3, Epoch 3 성능 평가:
Loss: 0.0121 | Acc: 0.9956 | F1: 0.9929 | Precision: 0.9859 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9979
Run 3: New best model found with combined score: 0.9979
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.99

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.8236 | Acc: 0.8564 | F1: 0.7082 | Precision: 0.9965 | Recall: 0.5492 | AUROC: 0.9845 | TPR@FPR=0.01%: 0.2701
소요 시간: 126.60분


In [ ]:
# kobert 출력

tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model_class = AutoModelForSequenceClassification
model_name = "monologg/kobert"

커리큘럼 방식 실험

In [ ]:
# predefined1


result_dir = f"/content/drive/My Drive/cmf/"
safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "predefined1"



class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.difficulty_score = dataframe['difficulty_score1'].tolist()  #difficulty_score1 difficulty_score2 difficulty_score3
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }

def re_weighting_sequence(target_series, current_epoch, total_epoch=5):

    easy_threshold = min(1.0, current_epoch*1.0/total_epoch)

    hard_weight = 0.2 + (current_epoch / total_epoch) * 0.8
    res_series = [1 if e <= easy_threshold else hard_weight for e in target_series] # 점진적으로 어려운 샘플에 더 높은 가중치 부여

    return res_series


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader

def loader_return(dataframe, tokenizer, current_epoch, total_epoch, batch_size=16):
    target_col = dataframe['difficulty_score1']
    weights = re_weighting_sequence(target_col, current_epoch=current_epoch, total_epoch=total_epoch)

    dataset = TextDataset(dataframe, tokenizer)
    sampler = WeightedRandomSampler(weights=weights, num_samples=len(dataset), replacement=True)

    dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)

    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        train_dataloader_current = loader_return(train_data, tokenizer, epoch + 1, num_epochs)

        for step, batch in enumerate(train_dataloader_current):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():

    train_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_train.csv")
    val_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_val.csv")
    test_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_test.csv")


    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====
-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.7842 | Acc: 0.3075 | F1: 0.4704 | Precision: 0.3075 | Recall: 1.0000 | AUROC: 0.5772 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.0214 | Acc: 0.9956 | F1: 0.9929 | Precision: 0.9866 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9514
Run 1: New best model found with combined score: 0.9514
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9514 (weight: 1.00)
Run 1, Epoch 2 성능 평가:
Loss: 0.0047 | Acc: 0.9982 | F1: 0.9971 | Precision: 1.0000 | Recall: 0.9943 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9979
Run 1: New best model found with combined score: 0.9979
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9979 (weight: 1.00)
Run 1, Epoch 3 성능 평가:
Loss: 0.0150 | Acc: 0.9956 | F1: 0.9929 | Precision: 0.9908 | Recall: 0.9950 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9264
Run 1: No

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.7859 | Acc: 0.3075 | F1: 0.4704 | Precision: 0.3075 | Recall: 1.0000 | AUROC: 0.4649 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.0150 | Acc: 0.9965 | F1: 0.9943 | Precision: 0.9894 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9836
Run 2: New best model found with combined score: 0.9836
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9836 (weight: 1.00)
Run 2, Epoch 2 성능 평가:
Loss: 0.0065 | Acc: 0.9982 | F1: 0.9972 | Precision: 0.9943 | Recall: 1.0000 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.8536
Run 2: No improvement. Early stop counter = 1/2
Run 2, Epoch 3 성능 평가:
Loss: 0.0031 | Acc: 0.9996 | F1: 0.9993 | Precision: 0.9986 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9543
Run 2: No improvement. Early stop counter = 2/2
Run 2: Early stopping triggered at epoch 3
Run 2: Restored best model for final evaluation
----- Run 2: Final Evaluation on Test Data -----
Loss: 0.99

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.6340 | Acc: 0.6925 | F1: 0.0000 | Precision: 0.0000 | Recall: 0.0000 | AUROC: 0.5771 | TPR@FPR=0.01%: 0.0029
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.0107 | Acc: 0.9965 | F1: 0.9943 | Precision: 0.9922 | Recall: 0.9964 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9679
Run 3: New best model found with combined score: 0.9679
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9679 (weight: 1.00)
Run 3, Epoch 2 성능 평가:
Loss: 0.0051 | Acc: 0.9991 | F1: 0.9986 | Precision: 0.9993 | Recall: 0.9979 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9686
Run 3: New best model found with combined score: 0.9686
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9686 (weight: 1.00)
Run 3, Epoch 3 성능 평가:
Loss: 0.0058 | Acc: 0.9982 | F1: 0.9972 | Precision: 0.9943 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9957
Run 3: New best model found with combined score: 0.9957
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.99

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.5265 | Acc: 0.8867 | F1: 0.7844 | Precision: 0.9898 | Recall: 0.6496 | AUROC: 0.9867 | TPR@FPR=0.01%: 0.2103
소요 시간: 109.79분


In [ ]:
# predefined2


result_dir = f"/content/drive/My Drive/cmf/"
safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "predefined2"



class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.difficulty_score = dataframe['difficulty_score2'].tolist()  #difficulty_score1 difficulty_score2 difficulty_score3
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }

def re_weighting_sequence(target_series, current_epoch, total_epoch=5):

    easy_threshold = min(1.0, current_epoch*1.0/total_epoch)

    hard_weight = 0.2 + (current_epoch / total_epoch) * 0.8
    res_series = [1 if e <= easy_threshold else hard_weight for e in target_series] # 점진적으로 어려운 샘플에 더 높은 가중치 부여

    return res_series


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader

def loader_return(dataframe, tokenizer, current_epoch, total_epoch, batch_size=16):
    target_col = dataframe['difficulty_score2']
    weights = re_weighting_sequence(target_col, current_epoch=current_epoch, total_epoch=total_epoch)

    dataset = TextDataset(dataframe, tokenizer)
    sampler = WeightedRandomSampler(weights=weights, num_samples=len(dataset), replacement=True)

    dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)

    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        train_dataloader_current = loader_return(train_data, tokenizer, epoch + 1, num_epochs)

        for step, batch in enumerate(train_dataloader_current):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():

    train_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_train.csv")
    val_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_val.csv")
    test_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_test.csv")



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====
-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.7842 | Acc: 0.3075 | F1: 0.4704 | Precision: 0.3075 | Recall: 1.0000 | AUROC: 0.5772 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.0096 | Acc: 0.9980 | F1: 0.9968 | Precision: 0.9950 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9821
Run 1: New best model found with combined score: 0.9821
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9821 (weight: 1.00)
Run 1, Epoch 2 성능 평가:
Loss: 0.0043 | Acc: 0.9991 | F1: 0.9986 | Precision: 0.9979 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9857
Run 1: New best model found with combined score: 0.9857
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9857 (weight: 1.00)
Run 1, Epoch 3 성능 평가:
Loss: 0.0076 | Acc: 0.9985 | F1: 0.9975 | Precision: 0.9957 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9893
Run 1: Ne

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.8529 | Acc: 0.3077 | F1: 0.4704 | Precision: 0.3076 | Recall: 1.0000 | AUROC: 0.4700 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.0068 | Acc: 0.9982 | F1: 0.9971 | Precision: 0.9971 | Recall: 0.9971 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9836
Run 2: New best model found with combined score: 0.9836
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9836 (weight: 1.00)
Run 2, Epoch 2 성능 평가:
Loss: 0.0524 | Acc: 0.9780 | F1: 0.9655 | Precision: 0.9333 | Recall: 1.0000 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9736
Run 2: No improvement. Early stop counter = 1/2
Run 2, Epoch 3 성능 평가:
Loss: 0.0041 | Acc: 0.9987 | F1: 0.9979 | Precision: 1.0000 | Recall: 0.9957 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9964
Run 2: New best model found with combined score: 0.9964
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9964 (weight: 1.00)
Run 2, Epoch 4 성능 평가:
Loss: 0.0431 | Acc: 0.9853 | F1: 0.9766 | Pr

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.7678 | Acc: 0.3088 | F1: 0.4703 | Precision: 0.3076 | Recall: 0.9979 | AUROC: 0.3278 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.0197 | Acc: 0.9932 | F1: 0.9890 | Precision: 0.9790 | Recall: 0.9993 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9786
Run 3: New best model found with combined score: 0.9786
  - AUROC: 0.9999 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9786 (weight: 1.00)
Run 3, Epoch 2 성능 평가:
Loss: 0.0035 | Acc: 0.9991 | F1: 0.9986 | Precision: 0.9972 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9943
Run 3: New best model found with combined score: 0.9943
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9943 (weight: 1.00)
Run 3, Epoch 3 성능 평가:
Loss: 0.0296 | Acc: 0.9901 | F1: 0.9838 | Precision: 0.9892 | Recall: 0.9786 | AUROC: 0.9993 | TPR@FPR=0.01%: 0.9121
Run 3: No improvement. Early stop counter = 1/2
Run 3, Epoch 4 성능 평가:
Loss: 0.0142 | Acc: 0.9965 | F1: 0.9943 | Pr

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 1.4626 | Acc: 0.8005 | F1: 0.5418 | Precision: 0.9974 | Recall: 0.3719 | AUROC: 0.9599 | TPR@FPR=0.01%: 0.2538
소요 시간: 134.72분


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model_class = AutoModelForSequenceClassification
model_name = "monologg/kobert"


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


tokenization_kobert.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

In [ ]:
# predefined3

result_dir = f"/content/drive/My Drive/cmf/"
safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "predefined3"



class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.difficulty_score = dataframe['difficulty_score3'].tolist()  #difficulty_score1 difficulty_score3 difficulty_score3
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }

def re_weighting_sequence(target_series, current_epoch, total_epoch=5):

    easy_threshold = min(1.0, current_epoch*1.0/total_epoch)

    hard_weight = 0.2 + (current_epoch / total_epoch) * 0.8
    res_series = [1 if e <= easy_threshold else hard_weight for e in target_series] # 점진적으로 어려운 샘플에 더 높은 가중치 부여

    return res_series


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader

def loader_return(dataframe, tokenizer, current_epoch, total_epoch, batch_size=16):
    target_col = dataframe['difficulty_score3']
    weights = re_weighting_sequence(target_col, current_epoch=current_epoch, total_epoch=total_epoch)

    dataset = TextDataset(dataframe, tokenizer)
    sampler = WeightedRandomSampler(weights=weights, num_samples=len(dataset), replacement=True)

    dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)

    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        train_dataloader_current = loader_return(train_data, tokenizer, epoch + 1, num_epochs)

        for step, batch in enumerate(train_dataloader_current):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():

    train_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_train.csv")
    val_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_val.csv")
    test_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_test.csv")



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.7842 | Acc: 0.3075 | F1: 0.4704 | Precision: 0.3075 | Recall: 1.0000 | AUROC: 0.5772 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.0315 | Acc: 0.9925 | F1: 0.9877 | Precision: 0.9993 | Recall: 0.9764 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9700
Run 1: New best model found with combined score: 0.9700
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9700 (weight: 1.00)
Run 1, Epoch 2 성능 평가:
Loss: 0.0148 | Acc: 0.9963 | F1: 0.9940 | Precision: 0.9887 | Recall: 0.9993 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.5555
Run 1: No improvement. Early stop counter = 1/2
Run 1, Epoch 3 성능 평가:
Loss: 0.0030 | Acc: 0.9993 | F1: 0.9989 | Precision: 1.0000 | Recall: 0.9979 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9979
Run 1: New best model found with combined score: 0.9979
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9979 (weight: 1.00)
Run 1, Epoch 4 성능 평가:
Loss: 0.0052 | Acc: 0.9985 | F1: 0.9975 | Pr

...

New overall best model saved from run 1
  - Combined score: 0.9986
  - AUROC: 1.0000
  - TPR @ FPR=0.01%: 0.9986

===== Starting Run 2/3 =====
-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.8529 | Acc: 0.3077 | F1: 0.4704 | Precision: 0.3076 | Recall: 1.0000 | AUROC: 0.4700 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.0209 | Acc: 0.9952 | F1: 0.9921 | Precision: 1.0000 | Recall: 0.9843 | AUROC: 0.9996 | TPR@FPR=0.01%: 0.9843
Run 2: New best model found with combined score: 0.9843
  - AUROC: 0.9996 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9843 (weight: 1.00)
Run 2, Epoch 2 성능 평가:
Loss: 0.0168 | Acc: 0.9960 | F1: 0.9936 | Precision: 0.9880 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9843
Run 2: No improvement. Early stop counter = 1/2
Run 2, Epoch 3 성능 평가:
Loss: 0.0053 | Acc: 0.9985 | F1: 0.9975 | Precision: 0.9964 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9879
Run 2: New best model found with combined score: 0.9879

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.7678 | Acc: 0.3088 | F1: 0.4703 | Precision: 0.3076 | Recall: 0.9979 | AUROC: 0.3278 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.0072 | Acc: 0.9980 | F1: 0.9968 | Precision: 0.9950 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9900
Run 3: New best model found with combined score: 0.9900
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9900 (weight: 1.00)
Run 3, Epoch 2 성능 평가:
Loss: 0.0151 | Acc: 0.9956 | F1: 0.9929 | Precision: 0.9866 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9907
Run 3: New best model found with combined score: 0.9907
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9907 (weight: 1.00)
Run 3, Epoch 3 성능 평가:
Loss: 0.0061 | Acc: 0.9974 | F1: 0.9957 | Precision: 0.9915 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9986
Run 3: New best model found with combined score: 0.9986
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.99

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 1.2959 | Acc: 0.7796 | F1: 0.4674 | Precision: 1.0000 | Recall: 0.3050 | AUROC: 0.9798 | TPR@FPR=0.01%: 0.5951
소요 시간: 136.42분


In [ ]:
result_dir = f"/content/drive/My Drive/cmf/"


safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "adaptive"



class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }

def re_weighting_sequence(target_series, mode='adaptive'):
    wrong_cnt = np.sum(target_series)*1.0
    right_cnt = len(target_series)*1.0 - wrong_cnt
    if wrong_cnt == 0:
        wrong_cnt = 1
    if mode == 'adaptive':
        res_series = [right_cnt*1.0 / wrong_cnt if e == 1 else 1 for e in target_series]
    elif mode == 'static':
        res_series = [2 if e == 1 else 1 for e in target_series]

    return res_series

def wrong_first_dataloader(model, tokenized_datasets, batch_size=16, num_cls=2, current_epoch=1, device=torch.device("cpu")):
    if current_epoch == 1:
        return DataLoader(tokenized_datasets, shuffle=True, batch_size=batch_size)

    eval_dataloader = DataLoader(tokenized_datasets, shuffle=False, batch_size=1)

    all_preds = []
    all_labels = []
    right_wrong_list = []

    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )


        predictions = torch.argmax(outputs.logits, dim=-1)
        all_preds.append(predictions.item())
        all_labels.append(batch["labels"].item())

        right_wrong_list.append(1 if predictions.item() != batch["labels"].item() else 0)

    # 샘플링 가중치 설정
    weights = re_weighting_sequence(right_wrong_list)

    # 가중치 길이가 맞는지 확인
    if len(weights) != len(tokenized_datasets):
        print(f"Warning: weights length {len(weights)} does not match dataset size {len(tokenized_datasets)}")

    sampler = WeightedRandomSampler(weights=weights, num_samples=len(tokenized_datasets), replacement=True)

    return DataLoader(tokenized_datasets, batch_size=batch_size, sampler=sampler)




def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)


def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)
    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls)
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')


    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        train_dataloader_current = wrong_first_dataloader(
            model,
            train_dataloader.dataset,
            batch_size=train_dataloader.batch_size,
            num_cls=num_cls,
            current_epoch=epoch+1,
            device=device
        )

        for step, batch in enumerate(train_dataloader_current) :
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)



       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():

    train_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_train.csv")
    val_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_val.csv")
    test_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_test.csv")



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====
-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.7842 | Acc: 0.3075 | F1: 0.4704 | Precision: 0.3075 | Recall: 1.0000 | AUROC: 0.5772 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.0190 | Acc: 0.9932 | F1: 0.9888 | Precision: 0.9993 | Recall: 0.9786 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9743
Run 1: New best model found with combined score: 0.9743
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9743 (weight: 1.00)
Run 1, Epoch 2 성능 평가:
Loss: 0.0074 | Acc: 0.9980 | F1: 0.9968 | Precision: 1.0000 | Recall: 0.9936 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9957
Run 1: New best model found with combined score: 0.9957
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9957 (weight: 1.00)
Run 1, Epoch 3 성능 평가:
Loss: 0.0055 | Acc: 0.9980 | F1: 0.9968 | Precision: 0.9950 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9886
Run 1: No

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.6320 | Acc: 0.6925 | F1: 0.0000 | Precision: 0.0000 | Recall: 0.0000 | AUROC: 0.7396 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.0070 | Acc: 0.9978 | F1: 0.9964 | Precision: 0.9957 | Recall: 0.9971 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9900
Run 2: New best model found with combined score: 0.9900
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9900 (weight: 1.00)
Run 2, Epoch 2 성능 평가:
Loss: 0.0492 | Acc: 0.9859 | F1: 0.9777 | Precision: 0.9563 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9836
Run 2: No improvement. Early stop counter = 1/2
Run 2, Epoch 3 성능 평가:
Loss: 0.0297 | Acc: 0.9923 | F1: 0.9873 | Precision: 1.0000 | Recall: 0.9750 | AUROC: 0.9987 | TPR@FPR=0.01%: 0.9793
Run 2: No improvement. Early stop counter = 2/2
Run 2: Early stopping triggered at epoch 3
Run 2: Restored best model for final evaluation
----- Run 2: Final Evaluation on Test Data -----
Loss: 1.40

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



===== Starting Run 3/3 =====
-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.7283 | Acc: 0.3147 | F1: 0.4728 | Precision: 0.3097 | Recall: 0.9993 | AUROC: 0.6780 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.0460 | Acc: 0.9829 | F1: 0.9729 | Precision: 0.9472 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9957
Run 3: New best model found with combined score: 0.9957
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9957 (weight: 1.00)
Run 3, Epoch 2 성능 평가:
Loss: 0.0052 | Acc: 0.9989 | F1: 0.9982 | Precision: 0.9986 | Recall: 0.9979 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9836
Run 3: No improvement. Early stop counter = 1/2
Run 3, Epoch 3 성능 평가:
Loss: 0.0040 | Acc: 0.9989 | F1: 0.9982 | Precision: 0.9979 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9950
Run 3: No improvement. Early stop counter = 2/2
Run 3: Early stopping triggered at epoch 3
Run 3: Restored best model for final evaluation
----- Run 3: Final Evaluation

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 1.8280 | Acc: 0.7150 | F1: 0.1840 | Precision: 1.0000 | Recall: 0.1013 | AUROC: 0.7551 | TPR@FPR=0.01%: 0.2486
소요 시간: 112.59분


In [ ]:
# static

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "static_grd" #static

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }

def re_weighting_sequence(target_series, mode='static'):
    wrong_cnt = np.sum(target_series)*1.0
    right_cnt = len(target_series)*1.0 - wrong_cnt
    if wrong_cnt == 0:
        wrong_cnt = 1
    if mode == 'adaptive':
        res_series = [right_cnt*1.0 / wrong_cnt if e == 1 else 1 for e in target_series]
    elif mode == 'static':
        res_series = [2 if e == 1 else 1 for e in target_series]

    return res_series

def wrong_first_dataloader(model, tokenized_datasets, batch_size=16, num_cls=2, current_epoch=1, device=torch.device("cpu")):
    if current_epoch == 1:
        return DataLoader(tokenized_datasets, shuffle=True, batch_size=batch_size)

    eval_dataloader = DataLoader(tokenized_datasets, shuffle=False, batch_size=1)

    all_preds = []
    all_labels = []
    right_wrong_list = []

    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )


        predictions = torch.argmax(outputs.logits, dim=-1)
        all_preds.append(predictions.item())
        all_labels.append(batch["labels"].item())

        right_wrong_list.append(1 if predictions.item() != batch["labels"].item() else 0)

    # 샘플링 가중치 설정
    weights = re_weighting_sequence(right_wrong_list)

    # 가중치 길이가 맞는지 확인
    if len(weights) != len(tokenized_datasets):
        print(f"Warning: weights length {len(weights)} does not match dataset size {len(tokenized_datasets)}")

    sampler = WeightedRandomSampler(weights=weights, num_samples=len(tokenized_datasets), replacement=True)

    return DataLoader(tokenized_datasets, batch_size=batch_size, sampler=sampler)




def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)


def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)
    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls)
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')


    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []
    grad_norms_log = []
    epoch_stats_log = []  # 에폭별 통계

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화
        epoch_grad_norms = []

        train_dataloader_current = wrong_first_dataloader(
            model,
            train_dataloader.dataset,
            batch_size=train_dataloader.batch_size,
            num_cls=num_cls,
            current_epoch=epoch+1,
            device=device
        )

        for step, batch in enumerate(train_dataloader_current) :
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()
            total_norm = 0

            for p in model.parameters():
                if p.grad is not None:
                     param_norm = p.grad.data.norm(2)
                     total_norm += param_norm.item() ** 2

            total_norm = total_norm ** 0.5
            epoch_grad_norms.append(total_norm)
            grad_norms_log.append((epoch, step, total_norm))

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

        epoch_avg_norm = np.mean(epoch_grad_norms) if epoch_grad_norms else 0
        epoch_std_norm = np.std(epoch_grad_norms) if epoch_grad_norms else 0
        epoch_stats_log.append((epoch, epoch_avg_norm, epoch_std_norm))

        print(f"Epoch {epoch+1} - Gradient norm stats: Avg: {epoch_avg_norm:.4f}, Std: {epoch_std_norm:.4f}")

        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"


        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)



       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")

    # 그래디언트 노름 로그 저장
    grad_norms_df = pd.DataFrame(grad_norms_log, columns=['epoch', 'step', 'batch_norm'])
    grad_norms_df.to_csv(f"{result_dir}grad_norms_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8-sig')

    epoch_stats_df = pd.DataFrame(epoch_stats_log, columns=['epoch', 'avg_norm', 'std_norm'])
    epoch_stats_df.to_csv(f"{result_dir}epoch_grad_stats_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8-sig')


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    train_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_train.csv")
    val_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_val.csv")
    test_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_test.csv")


    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====


config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/559M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at lighthouse/mdeberta-v3-base-kor-further and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.7167 | Acc: 0.3075 | F1: 0.4704 | Precision: 0.3075 | Recall: 1.0000 | AUROC: 0.4199 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Epoch 1 - Gradient norm stats: Avg: 0.9385, Std: 2.2906
Run 2, Epoch 1 성능 평가:
Loss: 0.0292 | Acc: 0.9921 | F1: 0.9873 | Precision: 0.9749 | Recall: 1.0000 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.8329
Run 2: New best model found with combined score: 0.8329
  - AUROC: 0.9999 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.8329 (weight: 1.00)
Epoch 2 - Gradient norm stats: Avg: 0.2750, Std: 1.2879
Run 2, Epoch 2 성능 평가:
Loss: 0.0040 | Acc: 0.9991 | F1: 0.9986 | Precision: 0.9993 | Recall: 0.9979 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9857
Run 2: New best model found with combined score: 0.9857
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9857 (weight: 1.00)
Epoch 3 - Gradient norm stats: Avg: 0.0958, Std: 0.5676
Run 2, Epoch 3 성능 평가:
Loss: 0.0007 | Acc: 0.9998 | F1: 0.9996 | Precision: 1.0000 | R

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at lighthouse/mdeberta-v3-base-kor-further and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.6478 | Acc: 0.6925 | F1: 0.0000 | Precision: 0.0000 | Recall: 0.0000 | AUROC: 0.5938 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Epoch 1 - Gradient norm stats: Avg: 0.6569, Std: 1.6333
Run 3, Epoch 1 성능 평가:
Loss: 0.0149 | Acc: 0.9960 | F1: 0.9936 | Precision: 0.9873 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9314
Run 3: New best model found with combined score: 0.9314
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9314 (weight: 1.00)
Epoch 2 - Gradient norm stats: Avg: 0.3106, Std: 1.2050
Run 3, Epoch 2 성능 평가:
Loss: 0.0018 | Acc: 0.9993 | F1: 0.9989 | Precision: 0.9979 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9979
Run 3: New best model found with combined score: 0.9979
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9979 (weight: 1.00)
Epoch 3 - Gradient norm stats: Avg: 0.0057, Std: 0.0342
Run 3, Epoch 3 성능 평가:
Loss: 0.0020 | Acc: 0.9993 | F1: 0.9989 | Precision: 0.9979 | R

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at lighthouse/mdeberta-v3-base-kor-further and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.2812 | Acc: 0.9251 | F1: 0.8661 | Precision: 1.0000 | Recall: 0.7639 | AUROC: 0.9996 | TPR@FPR=0.01%: 0.7973
소요 시간: 370.47분


In [ ]:
# static
result_dir = f"/content/drive/My Drive/cmf/"

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "static"

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }

def re_weighting_sequence(target_series, mode='static'):
    wrong_cnt = np.sum(target_series)*1.0
    right_cnt = len(target_series)*1.0 - wrong_cnt
    if wrong_cnt == 0:
        wrong_cnt = 1
    if mode == 'adaptive':
        res_series = [right_cnt*1.0 / wrong_cnt if e == 1 else 1 for e in target_series]
    elif mode == 'static':
        res_series = [2 if e == 1 else 1 for e in target_series]

    return res_series

def wrong_first_dataloader(model, tokenized_datasets, batch_size=16, num_cls=2, current_epoch=1, device=torch.device("cpu")):
    if current_epoch == 1:
        return DataLoader(tokenized_datasets, shuffle=True, batch_size=batch_size)

    eval_dataloader = DataLoader(tokenized_datasets, shuffle=False, batch_size=1)

    all_preds = []
    all_labels = []
    right_wrong_list = []

    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )


        predictions = torch.argmax(outputs.logits, dim=-1)
        all_preds.append(predictions.item())
        all_labels.append(batch["labels"].item())

        right_wrong_list.append(1 if predictions.item() != batch["labels"].item() else 0)

    # 샘플링 가중치 설정
    weights = re_weighting_sequence(right_wrong_list)

    # 가중치 길이가 맞는지 확인
    if len(weights) != len(tokenized_datasets):
        print(f"Warning: weights length {len(weights)} does not match dataset size {len(tokenized_datasets)}")

    sampler = WeightedRandomSampler(weights=weights, num_samples=len(tokenized_datasets), replacement=True)

    return DataLoader(tokenized_datasets, batch_size=batch_size, sampler=sampler)




def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)


def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)
    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls)
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')


    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        train_dataloader_current = wrong_first_dataloader(
            model,
            train_dataloader.dataset,
            batch_size=train_dataloader.batch_size,
            num_cls=num_cls,
            current_epoch=epoch+1,
            device=device
        )

        for step, batch in enumerate(train_dataloader_current) :
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)



       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():

    train_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_train.csv")
    val_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_val.csv")
    test_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_test.csv")



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====


config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.6898 | Acc: 0.5581 | F1: 0.4779 | Precision: 0.3753 | Recall: 0.6579 | AUROC: 0.6274 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.0022 | Acc: 0.9993 | F1: 0.9989 | Precision: 0.9993 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9957
Run 1: New best model found with combined score: 0.9957
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9957 (weight: 1.00)
Run 1, Epoch 2 성능 평가:
Loss: 0.0009 | Acc: 0.9998 | F1: 0.9996 | Precision: 0.9993 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 1.0000
Run 1: New best model found with combined score: 1.0000
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 1.0000 (weight: 1.00)
Run 1, Epoch 3 성능 평가:
Loss: 0.0005 | Acc: 0.9998 | F1: 0.9996 | Precision: 0.9993 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 1.0000
Run 1: No improvement. Early stop counter = 1/2
Run 1, Epoch 4 성능 평가:
Loss: 0.0005 | Acc: 0.9998 | F1: 0.9996 | Pr

...

-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.6943 | Acc: 0.4964 | F1: 0.1331 | Precision: 0.1414 | Recall: 0.1257 | AUROC: 0.3188 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.0111 | Acc: 0.9971 | F1: 0.9954 | Precision: 0.9915 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9979
Run 2: New best model found with combined score: 0.9979
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9979 (weight: 1.00)
Run 2, Epoch 2 성능 평가:
Loss: 0.0033 | Acc: 0.9989 | F1: 0.9982 | Precision: 0.9971 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9979
Run 2: No improvement. Early stop counter = 1/2
Run 2, Epoch 3 성능 평가:
Loss: 0.0038 | Acc: 0.9991 | F1: 0.9986 | Precision: 0.9979 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9979
Run 2: No improvement. Early stop counter = 2/2
Run 2: Early stopping triggered at epoch 3
Run 2: Restored best model for final evaluation
----- Run 2: Final Evaluation on Test Data -----
Loss: 0.23

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.6989 | Acc: 0.3901 | F1: 0.4948 | Precision: 0.3320 | Recall: 0.9714 | AUROC: 0.6898 | TPR@FPR=0.01%: 0.0050
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.0049 | Acc: 0.9982 | F1: 0.9971 | Precision: 0.9971 | Recall: 0.9971 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9893
Run 3: New best model found with combined score: 0.9893
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9893 (weight: 1.00)
Run 3, Epoch 2 성능 평가:
Loss: 0.0052 | Acc: 0.9989 | F1: 0.9982 | Precision: 0.9993 | Recall: 0.9971 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9971
Run 3: New best model found with combined score: 0.9971
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9971 (weight: 1.00)
Run 3, Epoch 3 성능 평가:
Loss: 0.0016 | Acc: 0.9996 | F1: 0.9993 | Precision: 0.9986 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 1.0000
Run 3: New best model found with combined score: 1.0000
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 1.00

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.3062 | Acc: 0.9274 | F1: 0.8707 | Precision: 1.0000 | Recall: 0.7710 | AUROC: 0.9997 | TPR@FPR=0.01%: 0.8800
소요 시간: 74.86분


규제방식

In [ ]:
## kobert w dropout
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")

# 모델 config 가져오기
config = AutoConfig.from_pretrained("monologg/kobert")
config.hidden_dropout_prob = 0.3  # Dropout 비율 설정
config.attention_probs_dropout_prob = 0.1  # Attention Dropout 비율 설정
config.num_labels = 2

The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
## kobert w dropout

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "do"

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        for step, batch in enumerate(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가



def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = AutoModelForSequenceClassification.from_pretrained("monologg/kobert", config=config)

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():

    train_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_train.csv")
    val_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_val.csv")
    test_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_test.csv")



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


In [ ]:
# kobert 초기화

tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model_class = AutoModelForSequenceClassification
model_name = "monologg/kobert"

The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
# baseline w L1
result_dir = f"/content/drive/My Drive/cmf/"

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "L1"

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss

            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
#    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01) # L2 규제
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        for step, batch in enumerate(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
           # L1 추가
            l1_lambda = 0.01
            l1_penalty = sum(torch.norm(param, 1) for param in model.parameters() if param.requires_grad)
            loss += (l1_lambda * l1_penalty) / accumulation_steps
            # L1 추가

            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")

        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():

    train_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_train.csv")
    val_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_val.csv")
    test_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_test.csv")



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.7842 | Acc: 0.3075 | F1: 0.4704 | Precision: 0.3075 | Recall: 1.0000 | AUROC: 0.5772 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.1468 | Acc: 0.9794 | F1: 0.9667 | Precision: 0.9579 | Recall: 0.9757 | AUROC: 0.9970 | TPR@FPR=0.01%: 0.4307
Run 1: New best model found with combined score: 0.4307
Run 1, Epoch 2 성능 평가:
Loss: 0.1699 | Acc: 0.9479 | F1: 0.9219 | Precision: 0.8557 | Recall: 0.9993 | AUROC: 0.9977 | TPR@FPR=0.01%: 0.3321
Run 1: No improvement. Early stop counter = 1/2
Run 1, Epoch 3 성능 평가:
Loss: 0.1604 | Acc: 0.9695 | F1: 0.9522 | Precision: 0.9178 | Recall: 0.9893 | AUROC: 0.9963 | TPR@FPR=0.01%: 0.3600
Run 1: No improvement. Early stop counter = 2/2
Run 1: Early stopping triggered at epoch 3
Run 1: Restored best model for final evaluation
----- Run 1: Final Evaluation on Test Data -----
Loss: 0.4004 | Acc: 0.8246 | F1: 0.6539 | Precision: 0.8737 | Recall: 0.5225 | AUROC: 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.6373 | Acc: 0.6916 | F1: 0.0014 | Precision: 0.1667 | Recall: 0.0007 | AUROC: 0.5563 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.1071 | Acc: 0.9831 | F1: 0.9728 | Precision: 0.9629 | Recall: 0.9829 | AUROC: 0.9983 | TPR@FPR=0.01%: 0.3514
Run 2: New best model found with combined score: 0.3514
Run 2, Epoch 2 성능 평가:
Loss: 0.1196 | Acc: 0.9701 | F1: 0.9535 | Precision: 0.9147 | Recall: 0.9957 | AUROC: 0.9978 | TPR@FPR=0.01%: 0.5207
Run 2: New best model found with combined score: 0.5207
Run 2, Epoch 3 성능 평가:
Loss: 0.2220 | Acc: 0.9255 | F1: 0.8918 | Precision: 0.8061 | Recall: 0.9979 | AUROC: 0.9941 | TPR@FPR=0.01%: 0.0757
Run 2: No improvement. Early stop counter = 1/2
Run 2, Epoch 4 성능 평가:
Loss: 0.2870 | Acc: 0.8990 | F1: 0.8586 | Precision: 0.7535 | Recall: 0.9979 | AUROC: 0.9889 | TPR@FPR=0.01%: 0.0271
Run 2: No improvement. Early stop counter = 2/2
Run 2: Early stopping triggered

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.7315 | Acc: 0.3178 | F1: 0.4737 | Precision: 0.3105 | Recall: 0.9986 | AUROC: 0.5496 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.3529 | Acc: 0.9181 | F1: 0.8529 | Precision: 0.9524 | Recall: 0.7721 | AUROC: 0.9814 | TPR@FPR=0.01%: 0.1329
Run 3: New best model found with combined score: 0.1329
Run 3, Epoch 2 성능 평가:
Loss: 0.1137 | Acc: 0.9835 | F1: 0.9731 | Precision: 0.9770 | Recall: 0.9693 | AUROC: 0.9985 | TPR@FPR=0.01%: 0.4029
Run 3: New best model found with combined score: 0.4029
Run 3, Epoch 3 성능 평가:
Loss: 0.1237 | Acc: 0.9811 | F1: 0.9700 | Precision: 0.9475 | Recall: 0.9936 | AUROC: 0.9985 | TPR@FPR=0.01%: 0.4671
Run 3: New best model found with combined score: 0.4671
Run 3, Epoch 4 성능 평가:
Loss: 0.1462 | Acc: 0.9796 | F1: 0.9676 | Precision: 0.9437 | Recall: 0.9929 | AUROC: 0.9980 | TPR@FPR=0.01%: 0.5007
Run 3: New best model found with combined score: 0.5007
Run 3, Epoch 5 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.3316 | Acc: 0.8854 | F1: 0.8409 | Precision: 0.7511 | Recall: 0.9551 | AUROC: 0.9563 | TPR@FPR=0.01%: 0.0000
소요 시간: 111.21분


In [ ]:
# baseline w L2
result_dir = f"/content/drive/My Drive/cmf/"

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "L2"

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss

            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01) # L2 규제
#    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        for step, batch in enumerate(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
           # L1 추가
            # l1_lambda = 0.01
            # l1_penalty = sum(torch.norm(param, 1) for param in model.parameters() if param.requires_grad)
            # loss += (l1_lambda * l1_penalty) / accumulation_steps
            # L1 추가

            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")


    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():

    train_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_train.csv")
    val_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_val.csv")
    test_data = pd.read_csv("/content/drive/My Drive/25.1H Thesis/balanced_leave_one_out_gpt_4o_test.csv")



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.7842 | Acc: 0.3075 | F1: 0.4704 | Precision: 0.3075 | Recall: 1.0000 | AUROC: 0.5772 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.0134 | Acc: 0.9952 | F1: 0.9921 | Precision: 0.9993 | Recall: 0.9850 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9679
Run 1: New best model found with combined score: 0.9679
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9679 (weight: 1.00)
Run 1, Epoch 2 성능 평가:
Loss: 0.0053 | Acc: 0.9985 | F1: 0.9975 | Precision: 0.9979 | Recall: 0.9971 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9757
Run 1: New best model found with combined score: 0.9757
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9757 (weight: 1.00)
Run 1, Epoch 3 성능 평가:
Loss: 0.0082 | Acc: 0.9978 | F1: 0.9964 | Precision: 0.9943 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9893
Run 1: New best model found with combined score: 0.9893
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.98

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.6284 | Acc: 0.6927 | F1: 0.0029 | Precision: 0.6667 | Recall: 0.0014 | AUROC: 0.8445 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.0202 | Acc: 0.9949 | F1: 0.9918 | Precision: 0.9852 | Recall: 0.9986 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.3971
Run 2: New best model found with combined score: 0.3971
  - AUROC: 0.9998 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.3971 (weight: 1.00)
Run 2, Epoch 2 성능 평가:
Loss: 0.0029 | Acc: 0.9993 | F1: 0.9989 | Precision: 0.9986 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9993
Run 2: New best model found with combined score: 0.9993
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9993 (weight: 1.00)
Run 2, Epoch 3 성능 평가:
Loss: 0.0074 | Acc: 0.9974 | F1: 0.9957 | Precision: 0.9915 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9993
Run 2: No improvement. Early stop counter = 1/2
Run 2, Epoch 4 성능 평가:
Loss: 0.0056 | Acc: 0.9985 | F1: 0.9975 | Pr

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.6837 | Acc: 0.6293 | F1: 0.4715 | Precision: 0.4197 | Recall: 0.5379 | AUROC: 0.6359 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.0149 | Acc: 0.9960 | F1: 0.9936 | Precision: 0.9908 | Recall: 0.9964 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9721
Run 3: New best model found with combined score: 0.9721
  - AUROC: 0.9999 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9721 (weight: 1.00)
Run 3, Epoch 2 성능 평가:
Loss: 0.0063 | Acc: 0.9982 | F1: 0.9971 | Precision: 0.9957 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9964
Run 3: New best model found with combined score: 0.9964
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9964 (weight: 1.00)
Run 3, Epoch 3 성능 평가:
Loss: 0.0477 | Acc: 0.9886 | F1: 0.9818 | Precision: 0.9642 | Recall: 1.0000 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.4041
Run 3: No improvement. Early stop counter = 1/2
Run 3, Epoch 4 성능 평가:
Loss: 0.0061 | Acc: 0.9989 | F1: 0.9982 | Pr

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.9725 | Acc: 0.8378 | F1: 0.6573 | Precision: 0.9961 | Recall: 0.4904 | AUROC: 0.9921 | TPR@FPR=0.01%: 0.1262
소요 시간: 118.64분
